---------

# Enhanced BiGram model

----------

### Objective is correctness and clarity of concepts and not efficiency

#### Author : Anuj



------
------

IMPROVEMENTS:     
1) Better Tokenization      

LEFTOVERS:   

1) ~~Tensor board graphs for loss~~  

2) Tensor board for network vis      
   (a)   ~~Draw the Plot~~   
   (b)   Interpret and Map the network to Code     

3) Is embedding good or not ?    

4) Embedding visualization ?   
    (a) ~~Plot the vector in an interactive graph~~     
    (b) ~~Plot vectors in Tensorboard~~    
    (c) ~~After every k epochs, take some random words and show their nearest neighbours~~     
    (d) Show vectors being pulled appart in different direction    
    (e) Analogy tasks     
    (f) validation set loss   

5) model perplexity     

6) Input vectors and output vectors     

OPTIMIZATIONS     

1) Better Optimizer     
2) negative sampling    
3) manage learning rate     
4) Param initializations      
     

------
------


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import sys
import tensorflow as tf
import numpy as np
from collections import Counter
import random

In [3]:
sys.path.append("../../data/")


In [4]:
pwd

u'/Users/admin/Documents/Anuj/Coding/GitHub/Representation-Learning-for-NLP/module2/BiGram'

# Load the data file 

In [7]:
path = "./../../data/harry_potter_3.txt"
text = open(path).read()

raw_data = text.replace('\r\n', ' ')



In [6]:
tokens = raw_data.split()


## frequency counts

In [7]:
# count the frequency of tokens, pick top 100
counts = Counter(tokens)
top_100 = counts.most_common(100)

In [8]:
top_100

[('the', 4578),
 ('to', 2510),
 ('and', 2392),
 ('of', 1990),
 ('a', 1965),
 ('was', 1590),
 ('his', 1401),
 ('said', 1263),
 ('he', 1257),
 ('Harry', 1256),
 ('in', 1112),
 ('had', 865),
 ('at', 761),
 ('you', 717),
 ('--', 710),
 ('I', 706),
 ('it', 684),
 ('that', 667),
 ('on', 621),
 ('with', 620),
 ('as', 590),
 ('for', 487),
 ('He', 464),
 ('out', 427),
 ('Hermione', 402),
 ('Ron', 396),
 ('were', 385),
 ('they', 384),
 ('be', 376),
 ('him', 372),
 ('up', 356),
 ('from', 356),
 ('into', 353),
 ('Professor', 342),
 ('have', 331),
 ('but', 322),
 ('her', 321),
 ('--"', 319),
 ('The', 310),
 ('all', 285),
 ('back', 282),
 ('been', 264),
 ('very', 252),
 ('Harry,', 246),
 ('what', 245),
 ('"I', 241),
 ('who', 240),
 ('their', 237),
 ('looked', 234),
 ('them', 231),
 ('this', 222),
 ('down', 219),
 ('one', 214),
 ('like', 214),
 ('about', 214),
 ('she', 211),
 ('Lupin', 210),
 ('could', 209),
 ('got', 205),
 ('not', 201),
 ('an', 199),
 ('is', 199),
 ('over', 194),
 ('around', 190),
 

## Build token to Id mapping

In [9]:
word_to_id = {}

for token in set(tokens):
    word_to_id[token] = len(word_to_id)


In [10]:
id_to_word = dict(zip(word_to_id.values(), word_to_id.keys()))


# Convert Data to IDs

In [11]:
data = []

data = [word_to_id[tok] for tok in tokens]

In [12]:
print tokens[:20]

for id in data[:20]:
    print id_to_word[id] 

['Harry', 'Potter', 'and', 'the', 'Prisoner', 'of', 'Azkaban', 'by', 'J.K.', 'Rowling', 'CHAPTER', 'ONE', 'OWL', 'POST', 'Harry', 'Potter', 'was', 'a', 'highly', 'unusual']
Harry
Potter
and
the
Prisoner
of
Azkaban
by
J.K.
Rowling
CHAPTER
ONE
OWL
POST
Harry
Potter
was
a
highly
unusual


## Build validation set - randomly choose a batch from these 100

In [13]:
top_100_ids = []

for key, count in top_100:
    top_100_ids.append(word_to_id[key])
    
    
#print top_100_ids

In [14]:
validation_size = 32
validation_set = random.sample(top_100_ids, validation_size)

In [15]:
#top_100_ids

## Now lets slice the data into training set

In [16]:
class BatchData():
    
    def __init__(self, batch_size, list_of_token_ids):
        self.batch_size_ = batch_size
        self.data_ = list_of_token_ids
        self.data_len_ = len(self.data_)
        self.global_pointer = 0
        
    def getBatch(self):
         
        if self.global_pointer + self.batch_size_ > self.data_len_:
            print "Abort"
            
        X = np.zeros(shape=(self.batch_size_,), dtype=np.int32)
        Y = np.zeros(shape=(self.batch_size_,), dtype=np.int32)
        
        for i in range(self.batch_size_-1):
            
            X[i] = self.data_[self.global_pointer]
            Y[i] = self.data_[self.global_pointer+1]
            
            self.global_pointer = (self.global_pointer + 1)%self.data_len_
            
            
        return X, Y
    
    def reset(self):
        
        self.global_pointer = 0
    

In [17]:
#batch = BatchData(batch_size=32, list_of_token_ids=data)

In [18]:
#X, Y = batch.getBatch()

In [19]:
#print X
#print Y

In [20]:
#data[:64]

# Network

In [21]:
batch_size=32
num_batches = len(data)/batch_size

embedding_dims = 128
vocab_size = len(set(tokens))

In [22]:
X = tf.placeholder(shape=(batch_size,), dtype=tf.int32, name='X_var')
Y = tf.placeholder(shape=(batch_size,), dtype=tf.int32, name='Y_var')
valid_X = tf.Variable(validation_set, dtype=tf.int32, name='X_valid')

In [23]:
y_oh = tf.one_hot(indices=X, depth=vocab_size, name='Converting_Y_to_Y_oh')

In [24]:
print X.get_shape()
print Y.get_shape()
#print y_oh.get_shape()

(32,)
(32,)


In [25]:
embedding_layer_1 = tf.Variable(tf.truncated_normal(shape=(vocab_size, embedding_dims),mean=0.0, stddev=1.0, dtype=tf.float32), name="Embeddings_Matrix")    
embeded = tf.nn.embedding_lookup(embedding_layer_1, ids=X, name="Embedding_LookUp")

In [26]:
embeded.get_shape()

TensorShape([Dimension(32), Dimension(128)])

In [27]:
#softmax weights, bias
W = tf.Variable(tf.truncated_normal(shape=(embedding_dims, vocab_size),mean=0.0, stddev=1.0, dtype=tf.float32), name="Softmax_Weights_Matrix")
b = tf.Variable(tf.zeros(shape=(vocab_size,)), name="Softmax_Bias_Vector")


In [28]:
logits = tf.add(tf.matmul(embeded, W, name="WX"), b, name="WX_plus_b")

#logits = tf.add(tf.matmul(embed, softmax_weights, name="WX"), softmax_bias, name="WX_plus_b")

loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_oh, name="Compute_Loss")
#mean_loss = tf.reduce_mean(loss)
mean_loss = tf.reduce_mean(loss, name="Compute_mean_loss")

tf.summary.scalar("mean_loss", mean_loss)

<tf.Tensor 'mean_loss:0' shape=() dtype=string>

In [29]:
print logits.get_shape()
print y_oh.get_shape()

(32, 15519)
(32, 15519)


In [30]:
optimizer = tf.train.GradientDescentOptimizer(0.5, name="Optimizer").minimize(mean_loss)

In [31]:
summary_op = tf.summary.merge_all()

In [32]:
#compute L2 norm for cosine similarity
norm = tf.sqrt(tf.reduce_sum(tf.square(embedding_layer_1), axis=1, keep_dims=True))
normalised_embeddings = embedding_layer_1 / norm

# get validation set embeddings
validation_data_embeddings = tf.nn.embedding_lookup(normalised_embeddings, ids=valid_X, name="validation_embeddings_lookup")

# similarity score of validation embeddings w.r.t normalised= dot product between validation_data_embeddings and mornalised embeddings
similarity = tf.matmul(validation_data_embeddings, tf.transpose(normalised_embeddings))  # C.A = C x transpose(A)

# Training

In [33]:
num_of_epochs = 5
LOG_DIR = './bigram_chk_pts'

# A SIMPLE saver() to save the model
saver = tf.train.Saver()

batch = BatchData(batch_size=32, list_of_token_ids=data)

with tf.Session() as sess:
    
    # writer to write graph to tensorboard
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)

    tf.global_variables_initializer().run()
    print "initialised"

    for epoch_id in range(num_of_epochs):

        av_batch_loss = 0

        for batch_id in range(num_batches):

            X_, Y_ = batch.getBatch()

            feed_dict = {}
            feed_dict[X] = X_
            feed_dict[Y] = Y_

            batch_loss, _, summary = sess.run([mean_loss, optimizer,summary_op], feed_dict=feed_dict)
            
            #writer.add_summary(batch_loss, epoch) 
            writer.add_summary(summary, global_step=epoch_id)

            av_batch_loss += batch_loss
            
            if batch_id % 10000 == 0:
                print "For epoch = %d, batch id = %d, batch loss = %f" %(epoch_id, batch_id, batch_loss)
                
                #print validation data
                sim = similarity.eval() # compute similarity
                
                #iterate over each validation example
                for i in range(validation_size):
                    word = id_to_word[validation_set[i]]
                    top_k = 8
                    # sort indexes and pick top k. we take 1:top_k+1 since 0th top pick will the same word itself
                    nearest = (sim[i,:].argsort()[1:top_k+1])
                    log = '\t Nearest to %s : ' %word
                    for k in range(top_k):
                        nearest_word = id_to_word[nearest[k]]
                        log = '%s %s,' %(log, nearest_word)
                    print log
                        
                    
                    

        print "\nFor epoch = %d, Av loss = %f\n\n" %(epoch_id, av_batch_loss/num_batches)
        
        batch.reset()
        
    save_path = saver.save(sess, LOG_DIR)
    print("Model saved in file: %s" % save_path)
        
        
        
        

initialised
For epoch = 0, batch id = 0, batch loss = 32.577148
	 Nearest to is :  streamers, tidal, disordered,, striving, somewhere!", galloping., Parlor, yeh'll,
	 Nearest to when :  "Ron!"), 'Fortuna, odd, Hermes,, THAT, strong, absolutely, Prang.,
	 Nearest to Harry. :  Angelina., plunged, players);, scramble, clear., TRELAWNEY'S, feather, holidays,",
	 Nearest to looked :  help., hopelessly., allowed, Seeker!, Voldemort?, chess,, shrieked,, adopted,
	 Nearest to though :  "Riddikulus!", brings, important., classrooms, stated, lightest, whispered., listlessly.,
	 Nearest to the :  Ernie, dogs,", "Ah,", Hedwig.", complaints, inflating, He-Who-Must-Not-Be-Named?, eyes;,
	 Nearest to do :  tonic, huskily., be,", confiscated, desert, there?", frightening., people,,
	 Nearest to you :  "Lumos!", nor, paws...., fulfill, shuddering., thousands, holidays.", dormitory,,
	 Nearest to there :  Vernon,", Around, sundown, orphanage, other., vulture., COMFORTED, muck.,
	 Nearest to what :  shri


For epoch = 2, Av loss = 3.389148
For epoch = 3, batch id = 0, batch loss = 4.077061
	 Nearest to is :  Parlor, shield, galloping., Safe, striving, HERMIONE'S, Whichever, last....,
	 Nearest to when :  "Ron!"), Hermes,, all!", might,, 'Fortuna, odd, piggy, strong,
	 Nearest to Harry. :  plunged, scramble, Angelina., players);, clear., holidays,", BY, "Never,
	 Nearest to looked :  help., hopelessly., pervaded, Seeker!, reminding, silhouette., clench., her,
	 Nearest to though :  "Riddikulus!", brings, important., classrooms, stated, "For..., station,, whispered.,
	 Nearest to the :  Dumbledore's, Ernie, "Ah,", nervously., inflating, closed, Hedwig.", dogs,",
	 Nearest to do :  password, confiscated, shrilly, tonic, think,", search, subject,, be,",
	 Nearest to you :  fulfill, nor, "'Old, paws...., Slytherins, wheel., championship, dormitory,,
	 Nearest to there :  Vernon,", itself,, decay, other., ..., arm!", Around, vulture.,
	 Nearest to what :  shrivelfig,", sternlooking, Marge,, l

# Plot the Embeddings 

## Tensorboard way

In [34]:
#create metadata file

#reference: https://www.tensorflow.org/get_started/embedding_viz#metadata
LOG_DIR = './bigram_chk_pts'

file_name = LOG_DIR + '/metadata.tsv'
fp = open(file_name, 'w')

for i, v in enumerate(id_to_word):
    print id_to_word[i]
    
    string = str(i) + '_' + id_to_word[i] + '\n'
    fp.write(string)
    print string
    
    #if i > 5:
    #    break
        
fp.close()

raining
0_raining

shouted,
1_shouted,

shouted.
2_shouted.

PREDICTION
3_PREDICTION

yellow
4_yellow

four
5_four

hippogriff-baiting,"
6_hippogriff-baiting,"

Does
7_Does

clotted
8_clotted

hanging
9_hanging

conjuring
10_conjuring

screaming,
11_screaming,

screaming.
12_screaming.

though?"
13_though?"

conjure
14_conjure

Shunpike
15_Shunpike

"Ouch
16_"Ouch

course...
17_course...

Patronus."
18_Patronus."

remedies?"
19_remedies?"

Penelope!"
20_Penelope!"

month."
21_month."

better,
22_better,

sputter
23_sputter

"NO
24_"NO

flicking
25_flicking

sinking
26_sinking

spider.
27_spider.

fingers?
28_fingers?

spider,
29_spider,

wood,
30_wood,

sprint;
31_sprint;

wood.
32_wood.

Hit
33_Hit

EIGHTEEN
34_EIGHTEEN

foul
35_foul

fingers.
36_fingers.

uncertain
37_uncertain

fingers,
38_fingers,

bringing
39_bringing

pairs.
40_pairs.

writing.
41_writing.

pairs,
42_pairs,

see....
43_see....

accidents;
44_accidents;

wooden
45_wooden

tangle,
46_tangle,

"Good.
47_"Good.

pili

door.
478_door.

retreated,
479_retreated,

tersely.
480_tersely.

FACE
481_FACE

rat."
482_rat."

grip;
483_grip;

"THAT'S
484_"THAT'S

apiece.
485_apiece.

Slytherin!"
486_Slytherin!"

affair.
487_affair.

relaxation!
488_relaxation!

on!"
489_on!"

marvel
490_marvel

saying
491_saying

too."
492_too."

ponds.
493_ponds.

incredibly,
494_incredibly,

panted,
495_panted,

echoing
496_echoing

only-too-
497_only-too-

terror,
498_terror,

shaking,
499_shaking,

shaking.
500_shaking.

ecto
501_ecto

lessons
502_lessons

busy
503_busy

clicked
504_clicked

lazing
505_lazing

shuffling
506_shuffling

jerkily,
507_jerkily,

shape...
508_shape...

yellow-bellied
509_yellow-bellied

green;
510_green;

disembowelment
511_disembowelment

"Looks
512_"Looks

bush
513_bush

tight.
514_tight.

touched
515_touched

tight,
516_tight,

Snitch
517_Snitch

heartily
518_heartily

green,
519_green,

am...
520_am...

green.
521_green.

haven't,"
522_haven't,"

plate
523_plate

Headmaster,
524_Headmaster,


bit,"
1197_bit,"

Chris'mas.
1198_Chris'mas.

improving
1199_improving

such
1200_such

suck
1201_suck

first...
1202_first...

filthy
1203_filthy

robes."
1204_robes."

times
1205_times

Precisely
1206_Precisely

natural
1207_natural

mangy
1208_mangy

conscious
1209_conscious

move,"
1210_move,"

executioner!"
1211_executioner!"

darkened
1212_darkened

cannot
1213_cannot

so
1214_so

cloud.
1215_cloud.

swollen
1216_swollen

"Run.
1217_"Run.

pulled
1218_pulled

wastepaper
1219_wastepaper

Cadogan
1220_Cadogan

Peter.
1221_Peter.

lighter
1222_lighter

There,
1223_There,

Plastered
1224_Plastered

Stan's
1225_Stan's

below.
1226_below.

years
1227_years

course
1228_course

certainly,"
1229_certainly,"

early.
1230_early.

naturally,
1231_naturally,

Flitwick,
1232_Flitwick,

Flitwick.
1233_Flitwick.

boars,
1234_boars,

tore
1235_tore

solitary
1236_solitary

Divination!"
1237_Divination!"

"Ron!")
1238_"Ron!")

"Fortuna
1239_"Fortuna

did,
1240_did,

did.
1241_did.

troubles
1242_


don't,"
1759_don't,"

dare
1760_dare

booklist.
1761_booklist.

teapot?"
1762_teapot?"

"Woss
1763_"Woss

stranger
1764_stranger

paused,
1765_paused,

dying?"
1766_dying?"

house-elf
1767_house-elf

Yourself
1768_Yourself

scurrying
1769_scurrying

inter
1770_inter

"Weasley's
1771_"Weasley's

clap
1772_clap

punishment.
1773_punishment.

couldn't
1774_couldn't

amazed.
1775_amazed.

gaggle
1776_gaggle

Vernon's
1777_Vernon's

windfall,
1778_windfall,

'If
1779_'If

potion
1780_potion

"Nonsense!"
1781_"Nonsense!"

members,
1782_members,

thinks
1783_thinks

blood-flavored
1784_blood-flavored

four"
1785_four"

there'll
1786_there'll

fright.
1787_fright.

noon
1788_noon

seen?
1789_seen?

stroking
1790_stroking

neck.
1791_neck.

Charing
1792_Charing

Sirius!"
1793_Sirius!"

Lavender,
1794_Lavender,

mead
1795_mead

think.
1796_think.

waited.
1797_waited.

think,
1798_think,

waited,
1799_waited,

passages,
1800_passages,

fools,
1801_fools,

malevolent
1802_malevolent

"Explain,
1

2596_Shards

teacher...
2597_teacher...

sleeves
2598_sleeves

She's
2599_She's

wheeled
2600_wheeled

die,"
2601_die,"

everyone'd
2602_everyone'd

straight
2603_straight

OUCH!
2604_OUCH!

admire
2605_admire

forgetting
2606_forgetting

Knowing.
2607_Knowing.

pressed
2608_pressed

uncover
2609_uncover

leaning
2610_leaning

allowed...
2611_allowed...

Cold
2612_Cold

"Very
2613_"Very

Miss
2614_Miss

cages
2615_cages

winged,
2616_winged,

bag!"
2617_bag!"

Brandy?"
2618_Brandy?"

Foul?
2619_Foul?

hoping
2620_hoping

backing
2621_backing

need's
2622_need's

binding
2623_binding

sighed,
2624_sighed,

dangerous!
2625_dangerous!

sighed.
2626_sighed.

hover
2627_hover

Vector,
2628_Vector,

Foul.
2629_Foul.

owes
2630_owes

beautifully
2631_beautifully

vanish
2632_vanish

waist;
2633_waist;

chase
2634_chase

clucked
2635_clucked

funny
2636_funny

sounding
2637_sounding

since
2638_since

"Hello,
2639_"Hello,

office,"
2640_office,"

tells
2641_tells

shorter
2642_shorter

read
26

...

10920_Peter's

barrier.
10921_barrier.

quicker.
10922_quicker.

Freezing
10923_Freezing

cure.
10924_cure.

Protectively
10925_Protectively

white,
10926_white,

"Check
10927_"Check

home-baked
10928_home-baked

Herinione
10929_Herinione

Whomping
10930_Whomping

dyin'!"
10931_dyin'!"

minutes,"
10932_minutes,"

backward.
10933_backward.

DAD!"
10934_DAD!"

families.
10935_families.

approached
10936_approached

"Ha!"
10937_"Ha!"

retrieved
10938_retrieved

grunting
10939_grunting

You-Know-Who
10940_You-Know-Who

"Circumstances
10941_"Circumstances

Panicking,
10942_Panicking,

fourteen
10943_fourteen

salami
10944_salami

puffapod.
10945_puffapod.

lowered,
10946_lowered,

Magic!
10947_Magic!

illegally.
10948_illegally.

appeared,
10949_appeared,

collywobbles."
10950_collywobbles."

rope,
10951_rope,

Grade
10952_Grade

explanations
10953_explanations

appeared.
10954_appeared.

blank."
10955_blank."

"Hi,
10956_"Hi,

Silence,
10957_Silence,

wickerwork
10958_wickerwork

smitheree

along.
11890_along.

splashed
11891_splashed

expert,
11892_expert,

d'you
11893_d'you

stripping-down
11894_stripping-down

free
11895_free

Strutting
11896_Strutting

Rosmerta
11897_Rosmerta

went.
11898_went.

went,
11899_went,

formation
11900_formation

evil
11901_evil

W
11902_W

North
11903_North

question:
11904_question:

wanted
11905_wanted

Studies
11906_Studies

go,
11907_go,

go.
11908_go.

Marge?"
11909_Marge?"

CAT,
11910_CAT,

of."
11911_of."

go!
11912_go!

days
11913_days

priorities
11914_priorities

"this
11915_"this

go?
11916_go?

hopeless
11917_hopeless

go;
11918_go;

squinting
11919_squinting

password,
11920_password,

Never,
11921_Never,

onto
11922_onto

snowflakes,
11923_snowflakes,

YOU'RE
11924_YOU'RE

rang
11925_rang

already
11926_already

ticket."
11927_ticket."

half-moon
11928_half-moon

too!
11929_too!

too.
11930_too.

too,
11931_too,

alreadv
11932_alreadv

bandages
11933_bandages

last."
11934_last."

hearing
11935_hearing

"Angelina
11936_"Angel


injuries...."
12918_injuries...."

catching
12919_catching

"No
12920_"No

tumultuous
12921_tumultuous

means,"
12922_means,"

parchment,
12923_parchment,

reckons
12924_reckons

"Slytherin
12925_"Slytherin

MADE
12926_MADE

Sixty-ten.
12927_Sixty-ten.

contemptuous
12928_contemptuous

transformed,"
12929_transformed,"

Beats
12930_Beats

Eyes
12931_Eyes

too?"
12932_too?"

away."
12933_away."

wand...."
12934_wand...."

belt.
12935_belt.

EIGHT
12936_EIGHT

shouldn't
12937_shouldn't

meant....
12938_meant....

whispered,
12939_whispered,

whispered.
12940_whispered.

term?"
12941_term?"

END
12942_END

year?"
12943_year?"

askew.
12944_askew.

dispatched
12945_dispatched

gap.
12946_gap.

dog....
12947_dog....

shouted?"
12948_shouted?"

Peeves,"
12949_Peeves,"

trunk,
12950_trunk,

trunk.
12951_trunk.

sabotage!"
12952_sabotage!"

"Nope,"
12953_"Nope,"

wailed
12954_wailed

mended
12955_mended

more'n
12956_more'n

motion
12957_motion

turn
12958_turn

place
12959_place

retreating


13882_scuffled

thing."
13883_thing."

mastered
13884_mastered

stroked
13885_stroked

Dumblefore
13886_Dumblefore

trembling,
13887_trembling,

last-ditch
13888_last-ditch

We've
13889_We've

amazes
13890_amazes

weight
13891_weight

generation
13892_generation

flames.
13893_flames.

Dean.
13894_Dean.

Firebolt
13895_Firebolt

Mug
13896_Mug

"Even
13897_"Even

cloudy
13898_cloudy

Mum
13899_Mum

leaving....
13900_leaving....

Firebold
13901_Firebold

More
13902_More

impersonation
13903_impersonation

Scabbers
13904_Scabbers

wondered
13905_wondered

cleared;
13906_cleared;

edgy,
13907_edgy,

beast?"
13908_beast?"

health
13909_health

thunder,
13910_thunder,

SPEAK
13911_SPEAK

favoritism.
13912_favoritism.

"Welcome,"
13913_"Welcome,"

Bludger,
13914_Bludger,

tensely,
13915_tensely,

tensely.
13916_tensely.

MIDNIGHT...
13917_MIDNIGHT...

forgive
13918_forgive

Zonko
13919_Zonko

experimenting
13920_experimenting

madman,
13921_madman,

teach
13922_teach

'round
13923_'round

bus

14811_heads,

restrain
14812_restrain

eyeing
14813_eyeing

orange
14814_orange

clipping.
14815_clipping.

holiday
14816_holiday

world.
14817_world.

boy...
14818_boy...

world,
14819_world,

landing,
14820_landing,

happened,
14821_happened,

happened.
14822_happened.

badge.
14823_badge.

crash
14824_crash

look.
14825_look.

pulse.
14826_pulse.

Wouldn't
14827_Wouldn't

practice
14828_practice

excitedly;
14829_excitedly;

flew
14830_flew

happened?
14831_happened?

Can
14832_Can

emerald
14833_emerald

fizz,
14834_fizz,

excitedly.
14835_excitedly.

"WHO
14836_"WHO

did...."
14837_did...."

fled
14838_fled

Help
14839_Help

crowd.
14840_crowd.

lockjaw
14841_lockjaw

crowd,
14842_crowd,

night's
14843_night's

yesterday.
14844_yesterday.

yesterday,
14845_yesterday,

squeaked,
14846_squeaked,

feast
14847_feast

Only"),
14848_Only"),

"Change
14849_"Change

by,
14850_by,

"Are
14851_"Are

trap
14852_trap

paddling
14853_paddling

witch-hunts,
14854_witch-hunts,

tray
14855_tray



In [35]:
from tensorflow.contrib.tensorboard.plugins import projector
import os

summary_writer = tf.summary.FileWriter(LOG_DIR)

#get config params
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = embedding_layer_1.name  # this is where you connect your embedding tensor

# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)

# run your tensorboard
# tensorboard --logdir="./bigram_chk_pts" --port 6006